### Import data and declare hyperparameters

In [1]:
import numpy as np

from utils import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from data_loading import *

from standardized_residuals import StandardizedResiduals

seed_everything(42)

In [2]:
load_path = "../data/processed_data_3Dmin/casp.npz"

X, Y = load_data(load_path)

normalize = True
splits = [0.7, 0.1, 0.1, 0.1]

subsets = split_and_preprocess(X, Y, splits=splits, normalize=normalize)

x_train, y_train, x_calibration, y_calibration, x_test, y_test, x_stop, y_stop = subsets["X_train"], subsets["Y_train"], subsets["X_calibration"], subsets["Y_calibration"], subsets["X_test"], subsets["Y_test"], subsets["X_stop"], subsets["Y_stop"]

print("X_train shape:", x_train.shape, "Y_train shape:", y_train.shape)
print("X_cal shape:", x_calibration.shape, "Y_cal shape:", y_calibration.shape)
print("X_test shape:", x_test.shape, "Y_test shape:", y_test.shape)
print("X_stop shape:", x_stop.shape, "Y_stop shape:", y_stop.shape)


input_dim = x_train.shape[1]
output_dim = y_train.shape[1]

n_train = x_train.shape[0]
n_test = x_test.shape[0]
n_calibration = x_calibration.shape[0]
n_stop = x_stop.shape[0]

dtype = torch.float32

x_train_tensor = torch.tensor(x_train, dtype=dtype)
y_train_tensor = torch.tensor(y_train, dtype=dtype)
x_stop_tensor = torch.tensor(x_stop, dtype=dtype)
y_stop_tensor = torch.tensor(y_stop, dtype=dtype)
x_calibration_tensor = torch.tensor(x_calibration, dtype=dtype)
y_calibration_tensor = torch.tensor(y_calibration, dtype=dtype)
x_test_tensor = torch.tensor(x_test, dtype=dtype)
y_test_tensor = torch.tensor(y_test, dtype=dtype)

alpha = 0.1

X_train shape: (32010, 7) Y_train shape: (32010, 3)
X_cal shape: (4573, 7) Y_cal shape: (4573, 3)
X_test shape: (4574, 7) Y_test shape: (4574, 3)
X_stop shape: (4573, 7) Y_stop shape: (4573, 3)


### Train the models

In [3]:
seed_everything(42)

hidden_dim = 128
num_layers = 3
batch_size = 32
num_epochs = 30
lr = 1e-3

trainloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor), batch_size= batch_size, shuffle=True)
stoploader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_stop_tensor, y_stop_tensor), batch_size= batch_size, shuffle=True)

standardized_residuals = StandardizedResiduals(input_dim, 
                            output_dim,
                            hidden_dim = hidden_dim,
                            num_layers = num_layers
                            )

standardized_residuals.fit(trainloader, 
                    stoploader,
                    num_epochs=num_epochs,
                    lr=lr,
                    verbose = 2
                    )

/Users/sbraun/Desktop/Gaussian_Conformal_Prediction_V2/code/standardized_residuals.py:266: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:2259.)
  z = torch.triangular_solve(diff, L, upper=False)[0]


Epoch 1: Avg NLL Loss = -0.4964 -- Stop loss: -0.6929 -- Best Stop Loss: inf
Epoch 2: Avg NLL Loss = -0.7229 -- Stop loss: -0.8217 -- Best Stop Loss: -0.692947858488643
Epoch 3: Avg NLL Loss = -0.8363 -- Stop loss: -0.8878 -- Best Stop Loss: -0.8216597942205576
Epoch 4: Avg NLL Loss = -0.8327 -- Stop loss: -0.8242 -- Best Stop Loss: -0.8877824727263484
Epoch 5: Avg NLL Loss = -0.9041 -- Stop loss: -0.9616 -- Best Stop Loss: -0.8877824727263484
Epoch 6: Avg NLL Loss = -0.9206 -- Stop loss: -0.8484 -- Best Stop Loss: -0.9615701879863139
Epoch 7: Avg NLL Loss = -0.9504 -- Stop loss: -0.7536 -- Best Stop Loss: -0.9615701879863139
Epoch 8: Avg NLL Loss = -0.9705 -- Stop loss: -1.0128 -- Best Stop Loss: -0.9615701879863139
Epoch 9: Avg NLL Loss = -0.9753 -- Stop loss: -0.9826 -- Best Stop Loss: -1.0127524775224965
Epoch 10: Avg NLL Loss = -0.9984 -- Stop loss: -0.8751 -- Best Stop Loss: -1.0127524775224965
Epoch 11: Avg NLL Loss = -1.0027 -- Stop loss: -1.0479 -- Best Stop Loss: -1.012752477

### Full output

In [4]:
standardized_residuals.conformalize(x=x_calibration_tensor, y=y_calibration_tensor, alpha = alpha)

5.511889457702637

In [5]:
coverage = standardized_residuals.get_coverage(x_test_tensor, y_test_tensor)
volumes  = standardized_residuals.get_average_volume(x_test_tensor)

print("Coverage:", coverage)
print("Average Volume:", volumes)

Coverage: 0.9121119379997253
Average Volume: 1.5569067001342773


### Revealed outputs

In [6]:
idx_knowned = np.array([0])

standardized_residuals.conformalize_revealed(idx_revealed = idx_knowned,
                                            x = x_calibration_tensor, 
                                            y = y_calibration_tensor, 
                                            alpha = alpha
                                            )

3.5451841354370117

In [7]:
coverage = standardized_residuals.get_coverage_revealed(x_test_tensor, y_test_tensor)
volumes  = standardized_residuals.get_average_volume_revealed(x_test_tensor, y_test_tensor[:, idx_knowned])

print("Coverage:", coverage)
print("Average Volume:", volumes)

TODO : checker si la definition du q_alpha reste la meme.
Coverage: 0.914516806602478
Average Volume: 1.5590709447860718


### Projection of the output

In [8]:
projection_matrix_tensor =  torch.randn((2, output_dim), dtype=dtype)

standardized_residuals.conformalize_projection(
                                            projection_matrix = projection_matrix_tensor,
                                            x = x_calibration_tensor, 
                                            y = y_calibration_tensor, 
                                            alpha = alpha
                                            )

4.126266956329346

In [9]:
coverage = standardized_residuals.get_coverage_projection(x_test_tensor, y_test_tensor)
volumes  = standardized_residuals.get_average_volume_projection(x_test_tensor)

print("Coverage:", coverage)
print("Average Volume:", volumes)

Coverage: 0.9112374186515808
Average Volume: 1.7222936153411865


### Missing outputs

In [10]:
# Add NaN values to the calibration and test sets

y_calibration_nan = add_nan(y_calibration, min_nan=1, max_nan=output_dim-1)
y_calibration_nan_tensor = torch.tensor(y_calibration_nan, dtype=dtype)

y_test_nan = add_nan(y_test, min_nan=1, max_nan=output_dim-1)
y_test_nan_tensor = torch.tensor(y_test_nan, dtype=dtype)

In [11]:
standardized_residuals.conformalize_missing(x = x_calibration_tensor,
                                            y = y_calibration_nan_tensor, 
                                            alpha = alpha
                                            )

0.8660470843315125

In [12]:
coverage_with_nan    = standardized_residuals.get_coverage_missing(x_test_tensor, y_test_nan_tensor)
coverage_full_vector = standardized_residuals.get_coverage(x_test_tensor, y_test_tensor)

print("Coverage with NaN:", coverage_with_nan)
print("Coverage full vector:", coverage_full_vector)

Coverage with NaN: 0.9079580307006836
Coverage full vector: 0.9121119379997253
